In [1]:
using Pkg
Pkg.activate("uwu")

  Activating new project at `/content/uwu`


In [2]:
Pkg.add("CUDSS")

   Resolving package versions...
  No Changes to `/content/uwu/Project.toml`
  No Changes to `/content/uwu/Manifest.toml`
Precompiling project...
   7747.1 ms  ✓ Atomix → AtomixCUDAExt
   9453.4 ms  ✓ CUDSS
  2 dependencies successfully precompiled in 19 seconds. 106 already precompiled.
  2 dependencies had output during precompilation:
┌ Atomix → AtomixCUDAExt
│  ┌ Warning: You are using CUDA 12.5.0 with a driver for CUDA 13.x.
│  │ It is recommended to upgrade your driver, or switch to automatic installation of CUDA.
│  └ @ CUDA ~/.julia/packages/CUDA/G7Cnt/src/initialization.jl:137
└  
┌ CUDSS
│  ┌ Warning: You are using CUDA 12.5.0 with a driver for CUDA 13.x.
│  │ It is recommended to upgrade your driver, or switch to automatic installation of CUDA.
│  └ @ CUDA ~/.julia/packages/CUDA/G7Cnt/src/initialization.jl:137
└  


In [ ]:
Pkg.add("CUDA")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


In [4]:
using CUDA, CUDA.CUSPARSE
using CUDSS
using SparseArrays, LinearAlgebra

LoadError: ArgumentError: Package CUDSS not found in current path.
- Run `import Pkg; Pkg.add("CUDSS")` to install the CUDSS package.

In [5]:
# CUDA.set_runtime_version!(local_toolkit=true)
CUDA.set_runtime_version!(v"13.0")

LoadError: IOError: open("/content/uwu/Project.toml", 65, 438): no such file or directory (ENOENT)

In [3]:
using CUDSS_jll
CUDSS_jll.host_platform

Linux x86_64 {cuda=13.0, cuda_local=false, cxxstring_abi=cxx11, julia_version=1.11.5, libc=glibc, libgfortran_version=5.0.0, libstdcxx_version=3.4.30}

In [4]:
 CUDA.versioninfo()

CUDA toolchain: 
- runtime 13.0, artifact installation
- driver 550.54.15 for 13.0
- compiler 13.0

CUDA libraries: 
- CUBLAS: 13.0.2
- CURAND: 10.4.0
- CUFFT: 12.0.0
- CUSOLVER: 12.0.4
- CUSPARSE: 12.6.3
- CUPTI: 2025.3.1 (API 13.0.1)
- NVML: 12.0.0+550.54.15

Julia packages: 
- CUDA: 5.9.1
- CUDA_Driver_jll: 13.0.1+1
- CUDA_Compiler_jll: 0.2.1+0
- CUDA_Runtime_jll: 0.19.1+0

Toolchain:
- Julia: 1.11.5
- LLVM: 16.0.6

Preferences:
- CUDA_Runtime_jll.version: 13.0

1 device:
  0: Tesla T4 (sm_75, 14.738 GiB / 15.000 GiB available)


In [3]:
T = Float32
n = 6
A_cpu = sprand(T, n, n, 0.005) + I
for i in 1:n
  A_cpu[i, 1:i-1] .= 0
end
x_cpu = zeros(T, n)
b_cpu = rand(T, n)

A_gpu = CuSparseMatrixCSR(A_cpu)
x_gpu = CuVector(x_cpu)
b_gpu = CuVector(b_cpu);

LoadError: UndefVarError: `sprand` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
Hint: a global variable of this name may be made accessible by importing SparseArrays in the current active module Main

In [36]:
@time sol_cpu = A_cpu \ b_cpu;
r_cpu = b_cpu - A_cpu * sol_cpu
@time norm(r_cpu)

  4.478154 seconds (667 allocations: 1001.206 MiB, 1.48% gc time, 0.92% compilation time)
  0.000039 seconds (1 allocation: 16 bytes)


1.1200948746484173e-8

In [37]:
length(nonzeros(A_cpu))

185772

In [38]:
CUDA.@time solver = CudssSolver(A_gpu, "G", 'F')
CUDA.@time cudss("analysis", solver, x_gpu, b_gpu)
CUDA.@time cudss("factorization", solver, x_gpu, b_gpu)
CUDA.@time cudss("solve", solver, x_gpu, b_gpu)

r_gpu = b_gpu - A_gpu * x_gpu
CUDA.@time norm(r_gpu)

  0.000038 seconds (36 CPU allocations: 864 bytes)
  0.266701 seconds (14 CPU allocations: 320 bytes)
  0.096636 seconds (118 CPU allocations: 7.492 KiB)
  0.003275 seconds (120 CPU allocations: 7.594 KiB)
  0.000145 seconds (25 CPU allocations: 464 bytes) (1 GPU allocation: 4 bytes, 11.08% memmgmt time)


86.04905f0